K-Means Using Spark

SET UP:Installing Spark and Importing Spark environment

# pySpark setup
# Installing PySpark in colab

In [1]:
!pip install pyspark
# Installing the PyDrive
!pip install -U -q PyDrive
# Installing java 8 
!apt install openjdk-8-jdk-headless -qq
# Setting the environment variable "JAVA_HOME". This installs the compiler in the required directory.
import os
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 199 kB 46.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=3a67112db67e01f237dfe80389ab404f2ce5b3d1cf6b7d5e706b59a0c8aa5289
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upg

Importing necessary Libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Importing pyspark which is the Python API for Apache Spark, containing the framework and set of libraries for large-scale data processing
import pyspark
# Importing PySpark SQL, which is a module in spark that integrates relational processing with Spark's functional programming API
from pyspark.sql import *
# Importing PySpark SQL Types class, which is a base class of all datatypes used to create Dataframes with a specific type.
from pyspark.sql.types import *
# Importing the list of built-in functions available for dataframes
from pyspark.sql.functions import *
# Importing SparkContext and SparkConf
from pyspark import SparkContext, SparkConf

Create a Session

In [4]:
conf= SparkConf().set("spark.ui.port", "4050")

In [5]:
#Create the context
Sc=pyspark.SparkContext (conf=conf)

In [6]:
spark= SparkSession.builder.getOrCreate()

In [7]:
# Setting up ngrok
# Downloading ngrok on linux subsystem for windows
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# Unzipping the ngrok file
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2022-10-20 16:22:14--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 54.237.133.81, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  18.8MB/s    in 0.7s    

2022-10-20 16:22:15 (18.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [8]:
# Importing the dataset
from sklearn.datasets import load_breast_cancer
breast_cancer= load_breast_cancer()

In [9]:
# Using PySpark, creating the schema from the dataset, and then printing the schema
pd_df = pd.DataFrame(breast_cancer['data'],columns=breast_cancer['feature_names'])
df= spark.createDataFrame(pd_df)
def set_df_columns_nullable(spark, df, column_list, nullable=False):
  for struct_feild in df.schema:
    if struct_feild.name in column_list:
      struct_feild.nullable = nullable
  df_mod = spark.createDataFrame(df.rdd, df.schema)
  return df_mod

df = set_df_columns_nullable(spark, df, df.columns)
df = df.withColumn('features', array(df.columns))
vectors = df.rdd.map(lambda row:Vectors.dense(row.features))
df.printSchema()                     
                               


root
 |-- mean radius: double (nullable = false)
 |-- mean texture: double (nullable = false)
 |-- mean perimeter: double (nullable = false)
 |-- mean area: double (nullable = false)
 |-- mean smoothness: double (nullable = false)
 |-- mean compactness: double (nullable = false)
 |-- mean concavity: double (nullable = false)
 |-- mean concave points: double (nullable = false)
 |-- mean symmetry: double (nullable = false)
 |-- mean fractal dimension: double (nullable = false)
 |-- radius error: double (nullable = false)
 |-- texture error: double (nullable = false)
 |-- perimeter error: double (nullable = false)
 |-- area error: double (nullable = false)
 |-- smoothness error: double (nullable = false)
 |-- compactness error: double (nullable = false)
 |-- concavity error: double (nullable = false)
 |-- concave points error: double (nullable = false)
 |-- symmetry error: double (nullable = false)
 |-- fractal dimension error: double (nullable = false)
 |-- worst radius: double (nullable

In [10]:
# Importing the linear algebra package from the the Machine Learning module in pyspark
from pyspark.ml.linalg import Vectors
# Creating a features dataframe in pyspark and a series for the target class
features = spark.createDataFrame(vectors.map(Row), ["features"])
labels = pd.Series(breast_cancer.target)

In [11]:
# Importing the clustering algorithm and the evaluator
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator


# trains a K-means model
Kmeans = KMeans().setK(2).setSeed(1)
model= Kmeans.fit(features)

#make predictions
predictions= model.transform(features)
#Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.8342904262826145
